# Explore Dataset

In [ ]:
import transformers
import sentencepiece as spm
import unicodedata
import sentencepiece
import re
import tqdm

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/kaggle/input/c4200m/C4_200M.tsv-00000-of-00010",sep = "\t",header = None, names = ["input","output"]) #lấy dự liệu đã được tiền xử lý từ code encoder - decoder
df = df[:1000000]
df.head()

In [ ]:
pd.set_option('display.max_colwidth', None)
print(df.iloc[15])
#in ra cầu đầu tiên bản full

In [ ]:
df.shape #dữ liệu gồm 18386522 dòng và 2 cột: Correct và error

In [ ]:
df.rename(columns={'correct': 'output', 'error': 'input'}, inplace=True)# đổi tên cột correct thành output và error thành input để đưa vào model


In [ ]:
df.head() # in ra 5 dòng đầu tiên của dữ liệu sau khi đã đổi tên cột

In [ ]:
dataset_size = df.shape #kích thước của dữ liệu. Bao gồm 289562
missing_values = df.isnull().sum() #kiểm tra số lượng giá trị null trong từng cột (output, input)

dataset_size, missing_values

In [ ]:
df.dropna(inplace=True) # loại bỏ các dòng có giá trị null
df.reset_index(drop=True, inplace=True) # reset lại index của dataframe

In [ ]:
#chuyển các câu sang dạng lowercase
df['input'] = df['input'].str.lower()
df['output'] = df['output'].str.lower()

In [ ]:
def decontracted(phrase): #hàm chuyển từ viết tắt thành từ đầy đủ
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"gon na", " going to", phrase)
    phrase = re.sub(r"wan na", " want to", phrase)
    phrase = re.sub(r"gonna", " going to", phrase)
    phrase = re.sub(r"wanna", " want to", phrase)


    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub("\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    return phrase

def clean_text(t): #hàm bỏ dấu và ký tự unicode

  t = unicodedata.normalize('NFKD', t).encode('ascii', 'ignore').decode('ascii') 
  t = decontracted(t)

  t = re.sub(r'x D', '', t)
  t = re.sub(r': D', '', t)
  t = re.sub(r': P', '', t)

  t = re.sub(r'xD', '', t)
  t = re.sub(r':D', '', t)
  t = re.sub(r':P', '', t)

  if '(' in t and ')' in t: #loại bỏ nội dung trong dấu ngoặc đơn => Hello (world) => Hello
    try:
      t = re.sub(t.split("(")[-1].split(")")[0], '', t)
    except:
      pass
    #t = re.sub("(", '', t)
    #t = re.sub(")", '', t)

  t = re.sub(r'[^A-Za-z;!?.,\-\' ]+', ' ', t) #loại bỏ các emoji, số, ký tự đặc biệt

#chuẩn hóa dấu câu, thêm khoảng trống trước dâu câu. 
  t = re.sub(r'\.+',r' .',t)
  t = re.sub(r'\;+',r' , ',t)
  t = re.sub(r'!+',r' !',t )
  t = re.sub(r'\?+',r' ?',t )
  t = re.sub(r'\-+',r' - ',t )
  t = re.sub(r'\,+',r' , ',t )
  t = re.sub(r'\'+',r" ' ",t)
  t = re.sub(' +', ' ', t)
  t = t.lower()

  return t #trả về văn bản được làm sạch

In [ ]:
df["input"] = df["input"].apply(clean_text)
df["output"] = df["output"].apply(clean_text)

In [ ]:
#tạo noisy dữ liệu
def create_noisy_data(df, noise_level=0.1):
    noisy_df = df.copy()
    for i in tqdm(range(len(noisy_df))):
        if random.random() < noise_level:
            # Randomly swap two words in the input sentence
            words = noisy_df.at[i, 'input'].split()
            if len(words) > 1:
                idx1, idx2 = random.sample(range(len(words)), 2)
                words[idx1], words[idx2] = words[idx2], words[idx1]
                noisy_df.at[i, 'input'] = ' '.join(words)
    return noisy_df
df = create_noisy_data(df, noise_level=0.1) #tạo noisy dữ liệu với tỷ lệ noise là 0.1

# Train

## Importing libraries

In [ ]:
import pandas as pd
from tqdm import tqdm # thư viện hỗ trợ hiển thị tiến trình của vòng lặp
import random

import nltk #thư viện giúp hỗ trợ các tách văn bản thành câu, từ, ký tự
nltk.download('punkt') 
from nltk.tokenize import sent_tokenize #sent_tokenize: tách văn bản thành câu

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

## Set seed

In [ ]:
def set_seed(seed): #hàm để thiết lập seed cho các thư viện random, numpy và torch
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)

## The dataset

In [ ]:
#T5ForConditionalGeneration: giúp load mô hình T5 đã được huấn luyện trước đó
#T5Tokenizer: giúp mã hóa và giải mã câu đầu vào và đầu ra
#tokenizer: nhận vào câu đầu vào => chuyển thành các token, từ token chuyển thành id
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer,
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )

from torch.utils.data import Dataset, DataLoader

In [ ]:
model_name = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name) #load tokenizer từ mô hình T5
model = T5ForConditionalGeneration.from_pretrained(model_name) #load mô hình T5 đã được huấn luyện trước đó
model.resize_token_embeddings(len(tokenizer))
print("Tokenizer vocab size:", len(tokenizer))
print("Model vocab size:", model.config.vocab_size)

In [ ]:
for params in model.encoder.block[:-6]:
    params.requires_grad = False

In [ ]:
df = df[:1000000]
print(df.shape) # in ra kích thước của dữ liệu sau khi đã cắt lại thành 2000000 dòng

In [ ]:
#loại bỏ các câu có độ dài sau khi token lớn hơn 40
def filter_long_inputs(dataset, tokenizer, max_length=40):
    filtered_dataset = []
    for _,example in dataset.iterrows():
        # Token hóa văn bản và tính độ dài
        length = len(tokenizer(example["input"]).input_ids)
        if length <= max_length:
            filtered_dataset.append(example)
    return pd.DataFrame(filtered_dataset)

filtered_dataset = filter_long_inputs(df, tokenizer, max_length=40)

print(filtered_dataset.shape)


In [ ]:
print(filtered_dataset.columns)

In [ ]:
print(filtered_dataset.info()) # in ra 5 dòng đầu tiên của dữ liệu sau khi đã lọc các câu có độ dài lớn hơn 256 token

In [ ]:
input_text = ["He loves football", "I usually eat ice-cream in the summer","Watching TV is my favorite hobby"]
for  text in input_text:
    print(f"Input text: {text}")
    tokenized_text = tokenizer.tokenize(text)
    print(f"Tokenized text: {tokenized_text}")
    print("="*50)

In [ ]:
#sử dụng tokenizer cho câu đầu tiên
input_text = filtered_dataset['input'][0] #lấy câu đầu vào đầu tiên từ dữ liệu
print("Input text:", input_text) #in ra câu đầu vào
print("Tokenized input:", tokenizer.tokenize(input_text)) #in ra các token của câu đầu vào
print("Tokenized input IDs:", tokenizer.encode(input_text)) #in ra các id của các token trong câu đầu vào
print(tokenizer(input_text, return_tensors="pt")) #in ra các tensor của câu đầu vào

In [ ]:
def calc_token_len(example): #hàm tính độ dài của câu đầu vào và đầu ra sau khi tokenized
    return len(tokenizer(example).input_ids)

In [ ]:
from sklearn.model_selection import train_test_split #sử dụng train_test_split để chia dữ liệu thành 2 phần: train và test
train_df, test_df = train_test_split(filtered_dataset, test_size=0.10, shuffle=True) # chia dữ liệu thành 2 phần: train và test với tỷ lệ 90% cho train và 10% cho test
train_df.shape, test_df.shape #in ra kích thước của dữ liệu train và test

In [ ]:
test_df['input_token_len'] = test_df['input'].apply(calc_token_len) #tính độ dài của câu đầu vào sau khi tokenized
train_df['input_token_len'] = train_df['input'].apply(calc_token_len) #tính độ dài của câu đầu vào sau khi tokenized
test_df.head() #in ra độ dài của  5 câu đầu vào đầu tiên sau khi tokenized

In [ ]:
print(test_df['input_token_len'].max()) #in ra độ dài lớn nhất của câu đầu vào sau khi tokenized

In [ ]:
test_df['input_token_len'].describe() #in ra thống kê về độ dài của câu đầu vào sau khi tokenized

In [ ]:
#chỉ giữ lại các câu có độ dài nhỏ hơn 40 token
test_df = test_df[test_df['input_token_len'] < 40] #giữ lại các câu có độ dài nhỏ hơn 40 token
test_df = test_df.reset_index(drop=True) #reset lại index của dataframe
#tương tự với train_df
train_df['input_token_len'] = train_df['input'].apply(calc_token_len) #tính độ dài của câu đầu vào sau khi tokenized
train_df = train_df[train_df['input_token_len'] < 40] #giữ lại các câu có độ dài nhỏ hơn 40 token

In [ ]:
print(test_df['input_token_len'].max()) #in ra độ dài lớn nhất của câu đầu vào sau khi tokenized
print(train_df['input_token_len'].max()) #in ra độ dài lớn nhất của câu đầu vào sau khi tokenized

In [ ]:
#lưu dataset train và test ra .npy
train_df.to_csv("train_df.csv", index=False) #lưu train_df vào file csv
test_df.to_csv("test_df.csv", index=False) #lưu test_df vào file csv

In [ ]:
train_df = pd.read_csv("/kaggle/working/train_df.csv")
test_df = pd.read_csv("/kaggle/working/test_df.csv")

In [ ]:
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_df) #tạo dataset dành cho train từ dataframe train_df
test_dataset = Dataset.from_pandas(test_df) #tạo dataset dành cho test từ dataframe test_df

In [ ]:
test_dataset #in ra các thuộc tính của test_dataset
print(test_dataset[0]) #in ra câu đầu vào và đầu ra của test_dataset

In [ ]:
print(len(train_dataset)) #in ra số lượng câu trong train_dataset
print(len(test_dataset)) #in ra số lượng câu trong test_dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader 
class GrammarDataset(Dataset): #lớp GrammarDataset: 
    def __init__(self, dataset, tokenizer,print_text=False): #nhận đầu vào là dataset, tokenizer
        self.dataset = dataset #dữ liệu đầu vào
        self.pad_to_max_length = False #không sử dụng padding thành max_length cho dữ liệu đầu vào
        self.tokenizer = tokenizer #nhận tokenizer
        self.print_text = print_text
        self.max_len = 40 #độ dài tối đa của câu đầu vào và đầu ra sau khi tokenized

    def __len__(self):
        return len(self.dataset) #trả về số câu của dataset


    def tokenize_data(self, example):
        input_, target_ = example['input'], example['output'] #input_ là câu đầu vào, target_ là câu đầu ra

        #tokenize câu đầu vào. paddding = False: không sử dụng padding cho câu đầu vào
        #max_length = self.max_len: nếu sau khi tokenized mà độ dài của câu đầu vào lớn hơn max_len thì sẽ cắt bớt đi để còn thành max_len
        #return_attention_mask=True: trả về attention mask cho câu đầu vào => giúp mô hình biết được đâu là padding và đâu là câu thật sự

        tokenized_inputs = tokenizer(input_, padding = False,
                                            max_length=self.max_len,
                                            truncation=True,
                                            return_attention_mask=True)

        tokenized_targets = tokenizer(target_, padding = False,
                                            max_length=self.max_len,
                                            truncation=True,
                                            return_attention_mask=True)

        inputs={"input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }
        #labels: là câu đầu ra sau khi tokenized
        #input_ids: là câu đầu vào sau khi tokenized
        #attention_mask: là mặt nạ attention cho câu đầu vào sau khi tokenized

        return inputs


    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index]) #lấy câu đầu vào và đầu ra thứu i

        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k])) #in ra độ dài của các thuộc tính trong inputs

        return inputs

In [ ]:
dataset = GrammarDataset(test_dataset, tokenizer, False)
for i in [120]:
    print(f"Input id: {dataset[i]['input_ids']}")
    print(f"Attention mask: {dataset[i]['attention_mask']}")
    print(f"Labels: {dataset[i]['labels']}")

## Evaluator

In [ ]:
!pip install evaluate
!pip install rouge_score

In [ ]:
import evaluate #thư viện tạo các ma trận đánh giá cho mô hình
rouge_metric = evaluate.load("rouge")

## Train Model

In [ ]:
#áp dụng data_collator cho dữ liệu đầu vào
#tokenizer: là tokenizer đã được load từ mô hình T5
#model: là mô hình T5 
#padding='longest': sử dụng padding cho câu đầu vào và đầu ra thành độ dài lớn nhất trong batch (batch_size lấy từ Trainer)
#return_tensors='pt': trả về tensor cho câu đầu vào và đầu ra
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding='longest', return_tensors='pt') 

In [ ]:
print(data_collator([dataset[100]])) #in ra dữ liệu đầu vào và đầu ra sau khi áp dụng data_collator
print(dataset[100])

In [ ]:
batch_size = 128
#seq2seq_trainer: là lớp huấn luyện mô hình T5
#args: là các tham số đầu vào cho mô hình T5
args = Seq2SeqTrainingArguments(output_dir="finetune_t5_latest_1_million", #thư mục lưu trữ mô hình đã được huấn luyện
                        eval_strategy="steps", #đánh giá mô hình sau mỗi batch
                        per_device_train_batch_size=batch_size, #batch_size cho train
                        per_device_eval_batch_size=batch_size, #batch_size cho eval
                        learning_rate=2e-5, #tốc độ học của mô hình
                        num_train_epochs=5, #số lượng epoch cho mô hình
                        weight_decay=0.01, #thêm vào hàm loss là weigth_decay* độ lớn của trọng số => giảm overfitting
                        save_total_limit=2, #giới hạn số lượng mô hình đã được lưu trữ
                        predict_with_generate=True, #sử dụng hàm generate để dự đoán đầu ra cho câu đầu vào
                        fp16 = True, #huấn luyệ với fp16 (sử dụng 16 bit để huấn luyện mô hình)
                        gradient_accumulation_steps = 6,
                        eval_steps = 250, #đánh giá mô hình sau mỗi 250 bước
                        save_steps = 250, #lưu trữ mô hình sau mỗi 250 bước
                        load_best_model_at_end=True, #sau khi train xong, tự động lưu lại checkpoint tốt nhất
                        logging_dir="/logs", #thư mục lưu trữ log
                        report_to="wandb") #sử dụng wandb để theo dõi quá trình huấn luyện mô hình

In [ ]:
import nltk
nltk.download('punkt')
from nltk.translate.gleu_score import sentence_gleu
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [nltk.sent_tokenize(pred.strip()) for pred in decoded_preds]
    decoded_labels = [nltk.sent_tokenize(label.strip()) for label in decoded_labels]

    gleu_scores = [sentence_gleu([ref], pred) for pred, ref in zip(decoded_preds, decoded_labels)]
    result = {"gleu": np.mean(gleu_scores) * 100}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
#bắt đầu train model
trainer = Seq2SeqTrainer(model=model, #truyền vào model cần train
                args=args, #truyền vào các tham số khi training cho model
                train_dataset= GrammarDataset(train_dataset, tokenizer), #truyền vào dữ liệu train đã được xử lý
                eval_dataset=GrammarDataset(test_dataset, tokenizer), #truyền vào dữ liệu test đã được xử lý
                processing_class = tokenizer, #truyền vào tokenizer đã được load từ mô hình T5
                data_collator=data_collator, #truyền vào data_collator đã được tạo ở trên
                compute_metrics=compute_metrics) #truyền vào hàm compute_metrics để đánh giá mô hình


In [ ]:
print("Tokenizer vocab size:", len(tokenizer))
print("Model vocab size:", model.config.vocab_size)


In [ ]:
#in ra token đặc biệt trong tokenizer
print(tokenizer.special_tokens_map) #in ra các token đặc biệt trong tokenizer

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("latestmodel")